In [1]:
# new plan, related to athlete crawling...
# I want to crawl through all relevant pages:
# games, sports, events (ind and team), and athletes

# another idea to fill in the db: take all the names (first and last individually)
# from the olympicdatabase site, and use those as search terms in olympic.org site. yay!
# do this after the full crawl for two reasons: both as a QC and way to complete the database,
# but also to see how well my algorithm did!

# when I start actually parsing results, I should have periodic dumps to file
# so the list of results doesn't grow out of control
# like, maybe every 100 results or something

# one of the errors that came up was from spaces in the link. Need to use urllib2.quote()

# info I have available:
# data about olympics in olympicsTable
# data about sports in sportsTable
# data about countries in countryTable
#
# data I will collect:
# event results in resultTable
# athlete data in AthleteTable

# also an effing different results style here:
# /london-2012/cycling-track/keirin-men

# need to redo olympics table...

In [2]:
# set up logging
import logging
reload(logging)
logger = logging.getLogger(__name__)

In [3]:
# set up imports here
import urllib2
import re
import json
from bs4 import BeautifulSoup
from bs4 import Tag
import sys
from datetime import datetime
import parse_functions as par
import os.path
from TimeoutClass import Timeout

In [4]:
# define page types
pOlympics = "olympics page"
pSport = "sport page"
pEvent = "individual event results page"
pTeamEvent = "team event results page"
pAthlete = "athlete page"
pOther = "other page"

In [5]:
# determine the type of page we are on
# take in a soup object and parse to determine 
# the type of page
def determine_page_type(soup):
    # use the page title (in head) to check the type
    title = pageSoup.find("title").string
    
    if re.match(re.compile(".*\d{4} \w{6} Olympics - results.*"),title):
        return pOlympics
    
    # see if it is an athlete page
    idSection = soup.find("section",class_="id-card")
    if idSection:
        idClass = idSection['class']
        if len(idClass)==3 and re.match(re.compile("noc-\w{3}"),idClass[2]):
            return pAthlete
    
    return pOther

In [6]:

def country_from_name(name):
    for c in countryTable:
        if countryTable[c]['name'].lower() == name.lower():
            return c

def sport_from_name(name):
    for c in sportsTable:
        if sportsTable[c]['name'].lower() == name.lower():
            return c        
        
def olympics_from_name(name):
    return name
    for c in olympicsTable:
        if olympicsTable[c]['name'].lower() == name.lower():
            return c     
      

In [7]:
# output files at regular intervals
def file_output(file_name, data):
    logger.info("Writing " + str(len(data)) + " to " + file_name)
    with open(file_name, 'w') as outfile:
        json.dump(data, outfile, indent=4, sort_keys=True, separators=(',', ':'))
        outfile.close()

In [8]:
# load all the sports
with open('./SportsTable.json') as data_file:
    sportsTable = json.load(data_file)
with open('./CountryIDTable.json') as data_file:
    countryIDTable = json.load(data_file)
with open('./CountryTable.json') as data_file:
    countryTable = json.load(data_file)

In [9]:
# functions to check for and load previous states

def check_for_file_list(filename):
    out = []
    if os.path.isfile(filename):
        with open(filename) as data_file:
            out = json.load(data_file)   
            logger.info("Loaded list file " + filename)
    else:
        logger.info("Created new list for file " + filename)
    return out

def check_for_file_dict(filename):
    out = {}
    if os.path.isfile(filename):
        with open(filename) as data_file:
            out = json.load(data_file)   
            logger.info("Loaded list file " + filename)
    else:
        logger.info("Created new list for file " + filename)
    return out

In [10]:
# testing

# define logging output
logging.basicConfig(filename='./FullCrawler.log',filemode='w',level=logging.DEBUG)
logger.info("--------------------------------------------------")
logger.info("--------------------------------------------------")
logger.info("Starting FullCrawler: " + str(datetime.now()))

# base url
baseUrl = "https://www.olympic.org"

# constants for link tracking
VISITED = "visited"
PROBLEM = "problem"

# seed with top level page
linksToVisit = check_for_file_list("LinksToVisit.json")
if len(linksToVisit) == 0:
    linksToVisit.append('/')
    logger.info("Seeding crawler with " + str(linksToVisit))

# count variables
interval = 1000
max_cnt = 1000000

# initialize output lists
# load them if available
visited = check_for_file_dict("Visited.json")
athletes = {}
results = []
rankings = []

# result counters
cnt = len(visited)

# big loop
while len(linksToVisit) > 0 and cnt<max_cnt:
    link = linksToVisit.pop()
    visited[link] = VISITED
    cnt = cnt+1
    if cnt%100 == 0:
        print cnt
        nVisits = len([k for k,v in visited.iteritems() if v==VISITED])
        nProblems = len([k for k,v in visited.iteritems() if v==PROBLEM])
        logger.info("Status report - successful visits so far: " + str(nVisits))
        logger.info("Status report - # links in to-visit list: " + str(len(linksToVisit)))
        logger.info("Status report - problems so far: " + str(nProblems))
    try:
        logger.info("page " + link + ": Starting parse")

        # get the html
        with Timeout(60):
            html = urllib2.urlopen(baseUrl+urllib2.quote(link)).read()

        # soup it up
        pageSoup = BeautifulSoup(html)

        # determine which type of page
        pageType = determine_page_type(pageSoup) 
        logger.info("page " + link + ": page type = " + pageType)

        # parse this page for new links
        # adding them to linksToVisit
        newLinks = par.parse_page(pageSoup)
        if len(newLinks) > 0:
            for l in newLinks:
                if (not l in visited) and (not l in linksToVisit): 
                    logger.info("---adding link " + l)
                    linksToVisit.append(l)

        # try to parse rankings, just returns if none found
        newRankings = par.parse_rankings(pageSoup)
        if newRankings:
            for rn in newRankings:
                if (rn['athlete'] == "team"):
                    rn['country'] = country_from_name(rn['country'])
                else:
                    if rn['country'] in countryIDTable:
                        rn['country'] = countryIDTable[rn['country']]
                    else:
                        logger.info("page " + link + ": adding new Country code: " + rn['country'])
                        countryIDTable[rn['country']] = "NEW COUNTRY"
                        
            rankings.extend(newRankings)

        # if it's an olympics page or an athlete page:
        if pageType == pAthlete:
            newAthlete = par.parse_athlete_page(pageSoup)

            # add new athlete info to athletes list
            athID = par.name_to_link(link.split("/")[-1])
            athletes[athID] = newAthlete
            athletes[athID]['link'] = link
            logger.info("page " + link + ": adding new athlete: " + athID)

            # get the results from this athlete page and add it to the results array     
            newResults = par.parse_results(pageSoup)
            if newResults:
                for res in newResults:
                    res['athlete'] = athID
                    sport = res['sport']

                    # add the sport if it's a new one
                    if (not sport in sportsTable):
                        logger.info("page " + link + ": adding new sport: " + sport)
                        sportsTable[sport] = {}
                        sportsTable[sport]['name'] = sport
                        sportsTable[sport]['events'] = {}
                    event = res['event']

                    # add the event if it's a new one
                    if (len(sportsTable[sport]['events']) == 0) or (not event in sportsTable[sport]['events']):
                        logger.info("page " + link + ": adding new event: " + sport + "/" + event)
                        sportsTable[sport]['events'][event] = event

                # add new results to array and update result count
                logger.info("page " + link + ": parsed " + str(len(newResults)) + " new results")
                results.extend(newResults)

        logger.info("page " + link + ": done with parse")

    # timeout error catch
    except Timeout.Timeout:
        logger.warning("page " + link + ": PROBLEM - http Timeout, adding to back of list")
        del visited[link]
        linksToVisit.insert(0,link)
        print link + " - PROBLEM: " + " http Timeout"
        
    # general error handling, log and track
    except Exception as err:
        logger.warning("page " + link + ": PROBLEM - " + format(err))
        visited[link] = PROBLEM
        print link + " - PROBLEM: " + format(err)
    
    
    # make log statement every 1000 pages and output files
    if cnt%interval == 0:
        logger.info(str(cnt) + " pages visited at " + str(datetime.now()))
    
        nextFile = "data/AthletesTable_" + str(cnt/interval) + ".json"
        file_output(nextFile, athletes)
        athletes = {}
            
        nextFile = "data/ResultsTable_" + str(cnt/interval) + ".json"
        file_output(nextFile, results)
        results = []
    
        nextFile = "data/RankingsTable_" + str(cnt/interval) + ".json"
        file_output(nextFile, rankings)
        rankings = []
        
        nextFile = "LinksToVisit.json"
        file_output(nextFile,linksToVisit)
        
        nextFile = "Visited.json"
        file_output(nextFile,visited)
        
        nextFile = "CountryIDTable.json"
        file_output(nextFile,countryIDTable)
        
        nextFile = "SportsTable.json"
        file_output(nextFile,sportsTable)
    
logger.info("Athlete crawl complete: " + str(datetime.now()))
print "DONE"

/sitecore/content/Home/page404?item=web%3a%7b8F246C4D-1CDB-46C6-87F0-4F1B09ECFEDB%7d%40en - PROBLEM: HTTP Error 404: Not Found
100
200
300
400
/dmitry-dorofeyev - PROBLEM: <urlopen error [Errno -2] Name or service not known>
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
/countries/inactif noc/russia - PROBLEM: HTTP Error 404: Not Found
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
/andreas-wellinger - PROBLEM: <urlopen error [Errno -2] Name or service not known>
3700
3800
3900
/klaus-sulzenbacher - PROBLEM: <urlopen error [Errno -2] Name or service not known>
4000
4100
4200
4300
/sitecore/service/notfound.aspx?item=web%3a%7b8F246C4D-1CDB-46C6-87F0-4F1B09ECFEDB%7d%40en - PROBLEM: HTTP Error 404: Not Found
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
/countries/inactif noc/unified team ex ussr - PROBLEM: HTTP Error 404: Not Found
6200
6300
6400
6500
6600
6700
6800
6900
/annick-pier

/usr/lib/python2.7/urllib.py:1288: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return ''.join(map(quoter, s))


In [ ]:
athletes

In [ ]:
# had problem here: /london-2012/cycling-track/team-sprint-women
# --- this is a team page with bracket style team results
# also issue with multiple copies of a link in the linksToVisit list

In [ ]:
with open('./Visited.json') as data_file:
    visited = json.load(data_file)
with open('./LinksToVisit.json') as data_file:
    linksToVisit = json.load(data_file)    
linksToVisit

In [ ]:
title = "Grenoble 1968 5000m men - Olympic Speed skating"
ol = par.name_to_link(re.compile("^\D+\d{4}").findall(title)[0])
ev = par.name_to_link(re.split("^\D+\d{4}",title)[1].split("-")[0])

print ol
print ev

In [ ]:
x = []
x.append(1)
x.append(2)
x.insert(0,3)
print x.pop()